In [ ]:
import os
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId

#### Importing dataset

In [ ]:
users_votes_df = pd.read_csv('./csvs/users_votes.csv')

In [ ]:
users_votes_df.info()

## Fetching votes of anomaly user

##### anomaly user id = 5ba4140431c74205dc97f182

In [ ]:
anomaly_user_df = users_votes_df[users_votes_df['user'] == '5ba4140431c74205dc97f182']
anomaly_user_df.info()

In [ ]:
# Sorting anomaly user votes in descending order of created_at
anomaly_user_new_df = anomaly_user_df.sort_values('created_at', ascending=False)
anomaly_user_new_df.info()

In [ ]:
# anomaly user votes except latest 300
anomaly_user_ex300_df = anomaly_user_new_df[300:]
anomaly_user_ex300_df.info() 

In [ ]:
anomaly_user_ex300_df.head()

In [ ]:
# To remove docs from users_votes collection
anomaly_user_id = list(anomaly_user_ex300_df['_id'])

In [ ]:
anomaly_user_id_list = []
for i in anomaly_user_id:
    anomaly_user_id_list.append(ObjectId(i))

In [ ]:
# To remove docs from wish_list collection
anomaly_user_youtube_id_list = list(anomaly_user_ex300_df['youtube_id'])

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

In [ ]:
users_votes = database['users_votes']
wishlist = database['wish_list']

#### Removing docs from users_votes collection requested by anomaly user except latest 300

In [ ]:
try:
    users_votes.delete_many({'_id': {'$in': anomaly_user_id_list}})
except Exception as e:
    print(e)

#### Updating votes for anomaly user in wishlist collection

In [ ]:
try:
    for youtube_id in anomaly_user_youtube_id_list:
        wishlist.update_many(
            {"youtube_id": youtube_id},
            {"$inc" : {"votes": -1}}
        );
except Exception as e:
    print(e)

#### Removing wishlist docs with 0 votes after the previous update

In [ ]:
pipeline = [
    {
        '$match': {
            'votes': {
                '$ne': 0
            }
        }
    }, {
        '$out': 'modified_wish_list'
    }
]

In [ ]:
try:
    wishlist.aggregate(pipeline)
except Exception as e:
    print(e)